In [5]:
import pandas as pd
import numpy as np

from keras.preprocessing.text import Tokenizer
from keras.engine.topology import Layer
from keras import initializers as initializers, regularizers, constraints
from keras.callbacks import Callback
from keras.layers import concatenate, GlobalMaxPooling1D, GlobalAveragePooling1D, SpatialDropout1D, Embedding, Input, Dense, LSTM, GRU, Bidirectional, TimeDistributed
from keras import backend as K
from keras.models import Model
import keras.layers as layers
from sklearn.metrics import roc_auc_score
from keras.preprocessing import sequence
from keras.models import Sequential
from keras.layers import Dense, Dropout, Embedding, LSTM, Bidirectional, Layer
from keras.datasets import imdb
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import *
from keras.models import Model
from keras import initializers, regularizers, constraints, optimizers, layers
from keras.callbacks import EarlyStopping, ModelCheckpoint
import numpy as np
import pandas as pd
import random
from dataset import *

data_files = [
    'D:/senior/aiCourse/dataSource/comment_classification/output/train.json']
vocab_file = 'D:/senior/aiCourse/dataSource/comment_classification/output/vocab.txt'
label_file = 'D:/senior/aiCourse/dataSource/comment_classification/labels.txt'
enb_file = 'D:/senior/aiCourse/dataSource/comment_classification/embedding/embedding.txt'
batch_size = 128
reverse = False
split_word = True
max_len = 400
data_files = ['../../../dataSource/comment_classification/output/validation.json']
vocab_file = '../../../dataSource/comment_classification/output/vocab.txt'
label_file = '../../../dataSource/comment_classification/labels.txt'
enb_file = '../../../dataSource/comment_classification/output/embedding.txt'
dataset1 = DataSet(data_files, vocab_file, label_file, batch_size,
                   reverse=reverse, split_word=split_word, max_len=max_len)


def dig_lists(l):
    output = []
    for e in l:
        if isinstance(e, list):
            output += dig_lists(e)
        else:
            output.append(e)
    return (output)


def pad_sequences(comment_to_id, maxlen, padding, truncating):
    features = np.zeros((len(comment_to_id), maxlen), dtype=int)
    for i, comment in enumerate(comment_to_id):
        if len(comment) <= maxlen and padding == 'pre':
            features[i, -len(comment):] = np.array(comment)[:maxlen]
        if len(comment) <= maxlen and padding == 'post':
            features[i, :len(comment)] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'post':
            features[i, :] = np.array(comment)[:maxlen]
        if len(comment) > maxlen and truncating == 'pre':
            features[i, :] = np.array(comment)[len(comment)-maxlen:]
    return features


def split_dataset(pad_comments, labels, split_frac):
    split_index = int(len(pad_comments)*split_frac)
    data_list = list(zip(pad_comments, labels))
    random.shuffle(data_list)
    pad_comments, labels = zip(*data_list)
    x_train, x_test = pad_comments[:split_index], pad_comments[split_index:]
    y_train, y_test = labels[:split_index], labels[split_index:]
    return x_train, y_train, x_test, y_test


comment_to_id = [x[0] for x in dataset1._raw_data]
pad_comments = pad_sequences(
    comment_to_id, maxlen=max_len, padding='post', truncating='post')

labels = [x[2].flatten().tolist() for x in dataset1._raw_data]
x_train, y_train, x_test, y_test = split_dataset(
    pad_comments[:1000], labels, 0.8)

x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
print('Finish')

# vocab size:  50000
# vocab size:  20
# Start to preprocessing data...
# load data from ../../../dataSource/comment_classification/output/validation.json ...
# Got 13444 data items with 105 batches
Finish


In [ ]:
def softmax(x, axis=1):
    """Softmax activation function."""
    ndim = K.ndim(x)
    if ndim == 2:
        return K.softmax(x)
    elif ndim > 2:
        e = K.exp(x - K.max(x, axis=axis, keepdims=True))
        s = K.sum(e, axis=axis, keepdims=True)
        return e / s
    else:
        raise ValueError('Cannot apply softmax to a tensor that is 1D')


densor1 = Dense(32, activation="tanh")
densor2 = Dense(2, activation="relu")
# We are using a custom softmax(axis = 1) loaded in this notebook
activator = Activation(softmax, name='attention_weights')
dotor = Dot(axes=1)


def one_step_attention(a):
    e = densor1(a)
    energies = densor2(e)
    alphas = activator(energies)
    context = dotor([alphas, a])
    return context


def get_model():

    inp = Input(shape=(400,))
    x = Embedding(50000, 256)(inp)
    x1 = Bidirectional(CuDNNGRU(100, return_sequences=True))(x)
    x2 = Bidirectional(CuDNNGRU(100, return_sequences=True))(x1)
    x3 = Bidirectional(CuDNNGRU(100, return_sequences=True))(x2)
    conc = concatenate([x1, x2, x3])
    x = Dropout(0.5)(conc)
    context = one_step_attention(x)
    context = Flatten()(context)
    merged = Dropout(0.5)(context)
    preds = Dense(80, activation='relu')(merged)
    model = Model(inputs=[inp], outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    print(model.summary())
    return model


def get_model1():

    inp = Input(shape=(400,))
    x = Embedding(50000, 256)(inp)
    x1 = Bidirectional(CuDNNGRU(100, return_sequences=False))(x)

    preds = Dense(80, activation='relu')(x1)
    model = Model(inputs=[inp], outputs=preds)
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=[f1])
    print(model.summary())
    return model


model = get_model1()

print(model.summary())
history = model.fit(x_train, y_train, epochs=2,
                    validation_data=(x_test, y_test), verbose=1)

In [49]:
class Evaluation():
    def __init__(self,predictions,y_True):
        self.allNum = len(predictions)*80
        self.TP=0
        self.TN=0
        self.FP=0
        self.FN=0
        self.calculate(predictions,y_True)
        self.printing()
    def printing(self):
        print(self.TP,
        self.TN,
        self.FP,
        self.FN)
        
        
        print('Accuracy:',(self.TP+self.TN)/(self.allNum))
        Recall = (self.TP)/(self.TP+self.FN)
        Precision = (self.TP)/(self.TP+self.FP)
        print('Recall:',Recall)
        print('Precision:',Precision)
        print('F1:',(2*Recall*Precision)/(Precision+Recall))
    def calculate(self,predictions,y_True):
        answer ={0:[1,0,0,0],1:[0,1,0,0],2:[0,0,1,0],3:[0,0,0,1]}
        predictions_binary = []
        for item in predictions:
            answer_temp = []
            for i in range(0,80,4):
                print(item[i:i+4])
                print(np.argmax(item[i:i+4]))
                answer_temp.append(answer[np.argmax(item[i:i+4])])
            l2=self.dig_lists(answer_temp)
            predictions_binary.append(l2)
            
        counter=0
        for i,item in enumerate(predictions_binary):
            for j,num in enumerate (item):
                if num==y_True[i][j]:
                    if num ==1:
                        self.TP+=1
                    elif num ==0:
                        self.TN+=1
                elif num!=y_True[i][j]:
                    if num==1:
                        counter+=1
                    else:
                        counter-=1
                    if num ==1:
                        self.FP+=1
                    elif num ==0:
                        self.FN+=1
        print(counter) 
        
    def dig_lists(self,l):
        output = []
        for e in l:
            if isinstance(e, list):
                output += self.dig_lists(e)
            else:
                output.append(e)
        return(output)

In [50]:
Evaluation(preds,y_test)

[0.26511925 0.         0.11977036 0.34588394]
3
[0.2495762 0.        0.        0.3746294]
3
[0.2168059  0.03235032 0.02226727 0.461703  ]
3
[0.04455905 0.         0.05723515 0.4143727 ]
3
[0.28826794 0.         0.07997704 0.28215894]
0
[0.04079271 0.         0.         0.40751222]
3
[0.09097894 0.05100543 0.         0.37388608]
3
[0.         0.23196942 0.         0.3493644 ]
3
[0.24991031 0.         0.05790782 0.        ]
0
[0.250666   0.         0.02492413 0.4201396 ]
3
[0.30738565 0.         0.         0.35210347]
3
[0.25704205 0.12217642 0.         0.3816745 ]
3
[0.18393484 0.19229172 0.17017388 0.        ]
1
[0.31621236 0.03209369 0.05807426 0.3755737 ]
3
[0.         0.11886309 0.         0.3432188 ]
3
[0.39033172 0.2961476  0.0989626  0.19678834]
0
[0.22420008 0.         0.0543372  0.3688975 ]
3
[0.         0.02152087 0.         0.        ]
1
[0.05552895 0.30202124 0.07099716 0.        ]
1
[0.3030146  0.0439156  0.01407849 0.        ]
0
[0.2666475  0.         0.13776112 0.35110024

3
[0.31474078 0.         0.         0.3860993 ]
3
[0.26581317 0.14422639 0.         0.41026598]
3
[0.19180332 0.19033878 0.1768995  0.        ]
0
[0.3171723  0.03653982 0.05888989 0.39552256]
3
[0.         0.13454597 0.         0.3462966 ]
3
[0.40333462 0.336576   0.12646352 0.21886258]
0
[0.22245799 0.         0.03134941 0.40008596]
3
[0. 0. 0. 0.]
0
[0.08255728 0.32147354 0.08169236 0.        ]
1
[0.3315068  0.0558604  0.01857176 0.        ]
0
[0.12625176 0.         0.08548178 0.14804544]
3
[0.2325789  0.         0.         0.18476222]
0
[0.19696833 0.         0.01272232 0.33914515]
3
[0.08835712 0.         0.08447914 0.24054   ]
3
[0.13404936 0.         0.07105434 0.13265108]
0
[0.02515261 0.         0.         0.23554967]
3
[0.06560911 0.03167826 0.         0.19462815]
3
[0.         0.09039928 0.         0.18873401]
3
[0.18601689 0.         0.0738313  0.        ]
0
[0.22383612 0.         0.00923462 0.28174806]
3
[0.20156    0.         0.         0.16851632]
0
[0.19050677 0.02870667

[0.03567028 0.         0.         0.42824313]
3
[0.10861109 0.05761895 0.         0.36489898]
3
[0.         0.24618421 0.         0.34529063]
3
[0.23682138 0.         0.07138968 0.        ]
0
[0.24404053 0.         0.01565696 0.39407292]
3
[0.30289617 0.         0.         0.36638248]
3
[0.25089028 0.14293958 0.         0.39386523]
3
[0.2034722  0.20505458 0.17956048 0.        ]
1
[0.3238145  0.06232085 0.04561093 0.39493975]
3
[0.         0.12887329 0.         0.35915047]
3
[0.3880465  0.34167445 0.1098752  0.1988859 ]
0
[0.22052182 0.         0.04414224 0.37872505]
3
[0. 0. 0. 0.]
0
[0.0740518  0.33320308 0.06739841 0.        ]
1
[0.337231   0.05546781 0.01734304 0.        ]
0
[0.29924765 0.         0.13643059 0.38002786]
3
[0.25377873 0.         0.         0.4194936 ]
3
[0.2047131  0.02871329 0.00627138 0.4648517 ]
3
[0.02730206 0.         0.04343074 0.43284005]
3
[0.29473838 0.         0.05985066 0.3182401 ]
3
[0.05018141 0.         0.         0.4360573 ]
3
[0.10524812 0.05309995 0

3
[0.03590345 0.         0.         0.40220505]
3
[0.0759256  0.06816254 0.         0.35737512]
3
[0.         0.21643765 0.         0.33308682]
3
[0.24220785 0.         0.06647881 0.        ]
0
[0.2528311  0.         0.01297108 0.40738496]
3
[0.2953152 0.        0.        0.3516906]
3
[0.24831954 0.12293673 0.         0.3809158 ]
3
[0.18623734 0.19811705 0.1841606  0.        ]
1
[0.31400895 0.05627719 0.06679152 0.36688244]
3
[0.         0.12063929 0.         0.3313779 ]
3
[0.37924904 0.3207057  0.10771003 0.2001589 ]
0
[0.23501946 0.         0.05252927 0.3615311 ]
3
[0. 0. 0. 0.]
0
[0.05726444 0.3064601  0.08218843 0.        ]
1
[0.28907585 0.05247705 0.         0.        ]
0
[0.2832407 0.        0.1273078 0.3766449]
3
[0.24657667 0.         0.         0.40468735]
3
[0.21244791 0.01842106 0.03220011 0.4584233 ]
3
[0.04093563 0.         0.03998189 0.41186178]
3
[0.29702958 0.         0.07880449 0.31210005]
3
[0.03123782 0.         0.         0.41252133]
3
[0.10365909 0.04917274 0.     

1
[0.05481562 0.29894978 0.07733654 0.        ]
1
[0.31387684 0.05637443 0.0058765  0.        ]
0
[0.28278565 0.         0.13115536 0.36047998]
3
[0.25564635 0.         0.         0.39789996]
3
[0.2195686  0.0292796  0.00184856 0.4627712 ]
3
[0.0503195  0.         0.03687627 0.43633384]
3
[0.29121184 0.         0.07075664 0.30540565]
3
[0.03108029 0.         0.         0.41369754]
3
[0.09566049 0.05015832 0.         0.37201115]
3
[0.         0.24148516 0.         0.34952536]
3
[0.24107857 0.         0.06657489 0.        ]
0
[0.2558098  0.         0.02559625 0.4260181 ]
3
[0.32304367 0.         0.         0.36835954]
3
[0.25424278 0.13185367 0.         0.39869982]
3
[0.20788354 0.20395014 0.15984198 0.        ]
0
[0.32796955 0.04431681 0.0511481  0.4022339 ]
3
[0.         0.14708261 0.         0.34976214]
3
[0.413445   0.31640846 0.1057132  0.20202461]
0
[0.20102552 0.         0.0370099  0.38904983]
3
[0.         0.01652595 0.         0.        ]
1
[0.05891345 0.32236573 0.07269641 0.  

...

[0.         0.25148186 0.         0.35537884]
3
[0.24375261 0.         0.04806566 0.        ]
0
[0.25156432 0.         0.00776871 0.40957436]
3
[0.31180102 0.         0.         0.37555203]
3
[0.25099403 0.143791   0.         0.37937978]
3
[0.19370346 0.2200939  0.18340635 0.        ]
1
[0.3090135  0.04605975 0.04331833 0.40802768]
3
[0.         0.12952112 0.         0.35206   ]
3
[0.4036355  0.3387811  0.12764476 0.22791165]
0
[0.22935657 0.         0.04153509 0.41831604]
3
[0.         0.00351307 0.         0.        ]
1
[0.08940116 0.3364095  0.05925419 0.        ]
1
[0.333429   0.07195748 0.00184914 0.        ]
0
[0.2646948  0.         0.12152356 0.33208594]
3
[0.254752   0.         0.         0.38225803]
3
[0.21883331 0.02961701 0.01748347 0.45517588]
3
[0.0723649  0.         0.04025069 0.40289748]
3
[0.2786926  0.         0.07341117 0.27640802]
0
[0.03087566 0.         0.         0.40407947]
3
[0.08296319 0.05804026 0.         0.3435949 ]
3
[0.        0.2308658 0.        0.3501008

[0.32905668 0.         0.         0.38831788]
3
[0.25302356 0.13961759 0.         0.3950279 ]
3
[0.19764547 0.20754707 0.18602502 0.        ]
1
[0.30785376 0.03770468 0.05885432 0.4022508 ]
3
[0.         0.12308206 0.         0.34788027]
3
[0.40024203 0.31560534 0.13215312 0.21289223]
0
[0.22318013 0.         0.04468364 0.41151837]
3
[0. 0. 0. 0.]
0
[0.0831003  0.32578585 0.06943998 0.        ]
1
[0.34092858 0.04894647 0.         0.        ]
0
[0.26733342 0.         0.11807777 0.34574607]
3
[0.24984244 0.         0.         0.3792551 ]
3
[0.22545001 0.02422448 0.03975    0.44844443]
3
[0.05455201 0.         0.06462605 0.3891768 ]
3
[0.27491546 0.         0.0802258  0.2794742 ]
3
[0.0400588  0.         0.         0.40758744]
3
[0.10123923 0.0522588  0.         0.35966274]
3
[0.         0.20859867 0.         0.3369273 ]
3
[0.25924757 0.         0.05800777 0.        ]
0
[0.2686744  0.         0.02871292 0.40462428]
3
[0.30128482 0.         0.         0.33517513]
3
[0.24123523 0.11673446 0

[0.15012386 0.09822285 0.14160967 0.        ]
0
[0.31492582 0.08621208 0.09787165 0.2690193 ]
0
[0.         0.08945744 0.         0.29244855]
3
[0.23396082 0.22639243 0.01742161 0.1381055 ]
0
[0.2286636  0.         0.10334591 0.21152619]
0
[0.        0.0761972 0.        0.       ]
1
[0.         0.18016441 0.06279101 0.        ]
1
[0.192599   0.02609969 0.01939006 0.        ]
0
[0.2872338 0.        0.1381267 0.3723777]
3
[0.25419867 0.         0.         0.4174086 ]
3
[0.21060713 0.05404309 0.00963269 0.45301706]
3
[0.05303461 0.         0.02492394 0.41822317]
3
[0.28544956 0.         0.06626021 0.29788584]
3
[0.02976481 0.         0.         0.4289588 ]
3
[0.09661398 0.05125023 0.         0.3645038 ]
3
[0.         0.24341606 0.         0.35302374]
3
[0.2266323  0.         0.06562556 0.        ]
0
[0.2516275 0.        0.0441014 0.4101337]
3
[0.31154472 0.         0.         0.37738913]
3
[0.25486463 0.12371256 0.         0.40612552]
3
[0.19695891 0.21106724 0.15418682 0.        ]
1
[0.3

3
[0.         0.00179765 0.         0.        ]
1
[0.06652348 0.2952208  0.07743679 0.        ]
1
[0.30673555 0.05885516 0.01314777 0.        ]
0
[0.1268574  0.         0.08556799 0.14412491]
3
[0.2219173  0.         0.         0.17823906]
0
[0.19278187 0.         0.02394922 0.31945524]
3
[0.07409964 0.         0.08175952 0.22900414]
3
[0.14118673 0.         0.08825438 0.1340449 ]
0
[0.02591219 0.         0.         0.21826237]
3
[0.05995923 0.03210216 0.         0.19481991]
3
[0.         0.09570873 0.         0.19636355]
3
[0.19572748 0.         0.0600965  0.        ]
0
[0.22212419 0.         0.01111705 0.2715565 ]
3
[0.18682094 0.         0.         0.1587072 ]
0
[0.16910365 0.03917938 0.         0.23568948]
3
[0.12812065 0.08765922 0.09302246 0.        ]
0
[0.23242785 0.0738413  0.06736606 0.20824893]
0
[0.         0.09813659 0.         0.22324789]
3
[0.13993013 0.15858117 0.01991199 0.06891327]
1
[0.22592244 0.         0.08225826 0.15570034]
0
[0.         0.07165772 0.         0.  

0
2259 10259 1741 1741
Accuracy: 0.782375
Recall: 0.56475
Precision: 0.56475
F1: 0.56475


In [21]:
def convert_2_labels(answer):
    labels_input = []
    for item in answer:
        answer_temp = []
        for i in range(0,80,4):
            answer_temp.append(np.argmax(item[i:i+4]))
        labels_input.append(answer_temp)
    return labels_input



In [24]:
y_test = convert_2_labels(y_test)

In [39]:
np.sum([1,2,3])

6

In [25]:
preds = convert_2_labels(preds)

In [44]:
def accuracy(predictions, labels):
    sum=0;
    for i,item in enumerate(predictions):
        for j,num in enumerate (item):
            if num==labels[i][j]:
                PT+=1
            else:
                
    return sum/(len(predictions)*20)

In [45]:
accuracy(preds,y_test)

0.49775

In [27]:
preds

[[3, 0, 0, 1, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 1, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [0, 0, 0, 0, 0, 3, 3, 3, 0, 0, 3, 3, 3, 3, 0, 1, 3, 0, 0, 0],
 [3, 3, 0, 1, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 1, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 1, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 1, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3, 0, 0],
 [3, 3, 0, 2, 0, 3, 3, 3, 0, 1, 3, 3, 1, 3, 0, 0, 3, 3,

In [31]:
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

np.array([0,1,0,1,0,1])
print(accuracy(np.array([[0,0,1,0,1,1,1]]),np.array([[0,0,1,0,0,0,0]])))

100.0


def get_model():
    embed_size = 256
    model = Sequential()   
    
    model.add( Embedding(50000, embed_size,input_length=400))
    
    
    model.add(Bidirectional(GRU(100, return_sequences=False)))
    #model.add(SpatialDropout1D(0.2))
              
    #model.add(Bidirectional(GRU(100, return_sequences=True)))
    
    #model.add(GlobalAveragePooling1D())
    model.add(Dense(80, activation="relu"))
    
    
    model.compile(loss='binary_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    return model

In [ ]:
import matplotlib.pyplot as plt
print(history.history.keys())
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')

plt.show()
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

preds = model.predict(x_train)
def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])
print(accuracy(preds,y_train))

In [ ]:
def convert_2_labels(answer):
    labels_input = []
    for item in answer:
        answer_temp = []
        for i in range(0,80,4):
            answer_temp.append(np.argmax(item[i:i+4]))
        labels_input.append(answer_temp)
    return labels_input



In [ ]:
convert_2_labels(preds)

In [ ]:
coun=0
all1=0
for line in convert_2_labels(y_train):
    for item in line:
        if item == 3:
            coun+=1
        all1+=1